<a href="https://colab.research.google.com/github/nermienkh/POC-Text-2-Text-Chatbot/blob/main/GPT2_Zero_Shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs

In [1]:
!nvidia-smi

Sat Jun 17 19:49:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:

!pip install transformers==4.19.2
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert
!pip install datasets
!pip install sacrebleu
!pip install fuzzysearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 96.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.3 MB/s eta 0:00:00
Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 34.80 MiB/s, done.
Resolving deltas: 100% (339/339), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.

# Imports

In [5]:
#textwrap enables formating of long text
import textwrap

from transformers import pipeline, GPT2TokenizerFast
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
from arabert.preprocess import ArabertPreprocessor

#you can choose any aragpt2 model since they all have the same preprocessing
arabert_processor = ArabertPreprocessor(model_name="aragpt2-mega")

In [6]:
import torch
device = 0 if torch.cuda.is_available() else -1
print(device)

0


# Get the Model

In [7]:
model_name = "aubmindlab/aragpt2-mega" #the mega model needs a High-Ram colab

grover_gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)
grover_gpt2_model.half()
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

aragpt2_pipeline = pipeline("text-generation",model=grover_gpt2_model, tokenizer=tokenizer,device=device)

Downloading:   0%|          | 0.00/923 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.49G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

# Zero Shot Translation

experimenting with the prompt

In [8]:
prompt = """
ما هي ترجمة هذه الجملة ?:
"""

In [9]:
text = """
\"I want to sleep\"
"""
text_tok = tokenizer.tokenize(arabert_processor.preprocess(text))
text_len = len(text_tok)
print(text_tok)
print(text_len)

['"', 'ĠI', 'Ġw', 'ant', 'Ġto', 'Ġs', 'le', 'ep', 'Ġ"']
9


In [10]:
text_prep = arabert_processor.preprocess(prompt + " " + text)
input_len = len(tokenizer.tokenize(text_prep))
print(text_prep)
print(input_len)

ما هي ترجمة هذه الجملة ? : " I want to sleep "
16


In [12]:
print("Input Length: ", input_len + text_len //2)
gen_text = aragpt2_pipeline(text_prep,
            pad_token_id=0, # 0 for AraGPT2
            do_sample = False,
            num_beams=5,
            max_length=int(input_len)+5,
            top_k=10,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3,
            num_return_sequences = 5)
gen_text[0]

Input Length:  20


{'generated_text': 'ما هي ترجمة هذه الجملة ? : " I want to sleep " ( أريد أن أنام )'}

In [ ]:
from datasets import load_dataset, load_metric
from tqdm.notebook import tqdm
import re

In [ ]:
en_ar_dataset = load_dataset("tatoeba", lang1="ar", lang2="en")

Dataset tatoeba downloaded and prepared to /root/.cache/huggingface/datasets/tatoeba/ar-en-lang1=ar,lang2=en/0.0.0/54423b66d13968ea583b6ac5828448a54b1a69944cabd3368ccd364fdb4f3216. Subsequent calls will reuse this data.


In [ ]:
en_ar_dataset_shuffled = en_ar_dataset['train'].shuffle(seed=42)
for data in en_ar_dataset_shuffled:
  print(data)
  break

{'id': '17747', 'translation': {'ar': 'كَم أحببت الكتاب الذي اشتريته بالأمس.', 'en': 'I liked the book which I bought yesterday very much.'}}


In [ ]:
len(en_ar_dataset['train'])

27596

In [ ]:
prompt = """
ما هي ترجمة هذه الجملة ?:
"""

In [ ]:
results = []
example_count = 5000
for idx, example in enumerate(tqdm(en_ar_dataset_shuffled, total=example_count)):
  ar_text = example['translation']['ar']
  en_text = example['translation']['en']

  en_text = re.sub("[^a-zA-Z ]","",en_text)

  ar_text_tok = tokenizer.tokenize(arabert_processor.preprocess(ar_text))
  ar_text_len = len(ar_text_tok)

  text_prep = arabert_processor.preprocess(prompt + " \"" + en_text + "\" ? ")
  input_len = len(tokenizer.tokenize(text_prep))

  gen_text = aragpt2_pipeline(text_prep,
            pad_token_id=0, # 0 for AraGPT2
            do_sample = False,
            num_beams=5,
            max_length=int(input_len + ar_text_len),
            top_k=10,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3,
            num_return_sequences = 1)[0]['generated_text']


  pred = gen_text[len(text_prep):]
  pred = re.sub("[^\u0621-\u063A\u0641-\u064A\u0660-\u0669\u0671-\u0673a-zA-Z ]","",pred)

  ar_target = arabert_processor.preprocess(ar_text)
  ar_target = re.sub("[^\u0621-\u063A\u0641-\u064A\u0660-\u0669\u0671-\u0673a-zA-Z ]","",ar_target)

  results.append({
      "orig_arabic" : ar_text,
      "orig_english": en_text,
      "input": text_prep,
      "generated_text": gen_text,
      "pred" : pred,
      "perp_arabic": ar_target
  })

  if idx==example_count:
    break

In [ ]:
len(results)

5001

Evaluation

In [ ]:
preds = [ x['pred'] for x in results]
targets = [ [x['perp_arabic']] for x in results]

In [ ]:
preds

[' What are you doing   ',
 '  هل أنت جائع  ',
 ' هل هذا يعني أنه يجب على',
 '  هذا هو قصصي عن',
 '   قصة',
 '  تعلم اللغة العربية',
 '  أنا أريد أن آكل ',
 '    هل',
 ' I  m not going to say that there is no doubt that the United States cannot help us for its people  and',
 ' الترجمة   أنت لا تعرف',
 '   إذا كنت',
 '   إذا',
 '    رابط   month',
 '   أهلا وسهلا بك زائرنا',
 'And what is this  What is it  How do you know  Who is thinking about your relationship  And who is going to look forward  Those who believe Israel  s struggle with',
 'أنت لم تسجل الدخول بعد أو أنك لا تملك صلاحية لدخول لهذه الصفحة  هذا قد يكون عائدا لأحد هذه الأسباب ليست لديك صلاحية أو إمتيازات كافية لدخول هذه الصفحة  هل تحاول تعديل مشاركة شخص آخر  دخول ميزات إدارية أو نظام متميز آخر ',
 '  أي ما الذي',
 ' الترجمة الحرفية لهذه الجملة هي  هذا',
 ' الترجمة بالعربي  ',
 '  أنا أحب أن أكون مع',
 ' أو ما هي الترجمة الحرفية لهذه الجملة ',
 ' I  m going to take advantage of some',
 ' الترجمة الأدبية من العربية  رؤية جديد

In [ ]:
metric = load_metric("sacrebleu")

In [ ]:
metric.add_batch(predictions=preds, references=targets)

In [ ]:
score = metric.compute()
score

{'bp': 0.8684929670288716,
 'counts': [2007, 428, 108, 27],
 'precisions': [8.986298916450256,
  2.4473924977127175,
  0.8271425289116949,
  0.29521102121145854],
 'ref_len': 25483,
 'score': 1.3221041145778176,
 'sys_len': 22334,
 'totals': [22334, 17488, 13057, 9146]}

# Zero-Shot question answering - TyDiQA

Start testing some prompts, to see what works

In [13]:
prompt = """
أجب عن السؤال التالي:
"""

In [14]:
text = """
\"من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟\"
"""
text_tok = tokenizer.tokenize(arabert_processor.preprocess(text))
text_len = len(text_tok)
print(text_tok)
print(text_len)

['"', 'ĠÙħÙĨ', 'ĠÙĥØ§ÙĨ', 'ĠØ±Ø¦ÙĬØ³', 'ĠØ£ÙĦÙħØ§ÙĨÙĬØ§', 'ĠØ§ÙĦÙĨØ§Ø²ÙĬØ©', 'ĠÙģÙĬ', 'ĠØ§ÙĦØŃØ±Ø¨', 'ĠØ§ÙĦØ¹Ø§ÙĦÙħÙĬØ©', 'ĠØ§ÙĦØ«Ø§ÙĨÙĬØ©', 'ĠØŁ', 'Ġ"']
12


In [15]:
text_prep = arabert_processor.preprocess(prompt + " " + text)
input_len = len(tokenizer.tokenize(text_prep))
print(text_prep)
print(input_len)

أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ "
18


In [16]:
print("Input Length: ", input_len + text_len //2)
gen_text = aragpt2_pipeline(text_prep,
            pad_token_id=0, # 0 for AraGPT2
            do_sample = False,
            num_beams=5,
            max_length=input_len + 10,
            top_k=10,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3,
            num_return_sequences = 5)
gen_text

Input Length:  24


[{'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " الجواب هو أدولف هتلر ، الذي حكم ألمانيا بين'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " الجواب هو أدولف هتلر ، الذي حكم ألمانيا لمدة'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " الجواب هو أدولف هتلر ، الذي حكم ألمانيا منذ'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " إن الإجابة على هذا السؤال صعبة جدا ، لأن هتلر'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " إن الإجابة على هذا السؤال صعبة للغاية ، لأن هتلر'}]

In [ ]:
text_prep = arabert_processor.preprocess("أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة")
input_len = len(tokenizer.tokenize(text_prep))
print(text_prep)
print(input_len)

أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة
17


In [ ]:
print("Input Length: ", input_len + text_len //2)
gen_text = aragpt2_pipeline("أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة",
            pad_token_id=0, # 0 for AraGPT2
            do_sample = False,
            num_beams=5,
            max_length=input_len + 4,
            top_k=10,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3,
            num_return_sequences = 5)
gen_text

Input Length:  23


[{'generated_text': 'أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة 1909 �'},
 {'generated_text': 'أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة 1906 �'},
 {'generated_text': 'أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة 1902 �'},
 {'generated_text': 'أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة 1904 �'},
 {'generated_text': 'أجب عن السؤال التالي : في أي سنة تأسست مدينة العيون ؟ الجواب هو في سنة 1903 �'}]

Once ready, download the TyDiQA test data to evaluate the prompt

In [ ]:
!wget https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-train.json
!wget https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-dev.json

--2021-02-24 13:53:45--  https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-train.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58004076 (55M) [application/json]
Saving to: ‘tydiqa-goldp-v1.1-train.json’

tydiqa-goldp-v1.1-t 100%[===================>]  55.32M  58.2MB/s    in 1.0s    

2021-02-24 13:53:47 (58.2 MB/s) - ‘tydiqa-goldp-v1.1-train.json’ saved [58004076/58004076]

--2021-02-24 13:53:47--  https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-dev.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 108.177.127.128, 172.217.218.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5617409 (5.4M) [applic

Preprocess, and load the data

In [ ]:
#@title
%%writefile prep_arabert_tydiqa.py
# coding=utf-8
#
# This script applies AraBERT's cleaning process and segmentation to ARCD or
# any SQUAD-like structured files and "naively" re-alligns the answers start positions

import tensorflow.compat.v1 as tf
from arabert.preprocess import ArabertPreprocessor
from arabert.arabert.tokenization import BasicTokenizer
from tqdm import tqdm
import json

from fuzzysearch import find_near_matches
from pyarabic import araby

flags = tf.flags
FLAGS = flags.FLAGS


## Required parameters
flags.DEFINE_string(
    "input_file", None, "The input json file with a SQUAD like structure."
)

flags.DEFINE_string(
    "output_file", None, "The ouput json file with AraBERT preprocessing applied."
)

flags.DEFINE_string(
    "model_name", None, "Model name same as HuggingFace library"
)


bt = BasicTokenizer(do_lower_case=False)

def clean_preprocess(text, processor):
    text = " ".join(
        bt._run_split_on_punc(text)
    )
    text= processor.preprocess(text)
    text = " ".join(text.split())  # removes extra whitespaces
    return text

def get_start_pos(old_context, old_answer_start,processor):
  new_context = clean_preprocess(old_context[:old_answer_start],processor)
  num_of_pluses = new_context.count("+")
  return old_answer_start + num_of_pluses*2 - 20

def normalize_alef(s):
    s = s.replace(araby.ALEF_HAMZA_ABOVE,araby.ALEF)
    s = s.replace(araby.ALEF_HAMZA_BELOW,araby.ALEF)
    s = s.replace(araby.ALEF_MADDA,araby.ALEF)
    return s

def main(_):
    tf.logging.set_verbosity(tf.logging.INFO)
    logger = tf.get_logger()
    logger.propagate = False

    print(FLAGS.model_name)
    arabert_prep = ArabertPreprocessor(model_name=FLAGS.model_name, remove_html_markup=False)


    with tf.gfile.Open(FLAGS.input_file, "r") as reader:
        input_data = json.load(reader)["data"]

    new_answers_count = 0
    no_answers_found_count = 0
    trunc_ans_count = 0
    for entry in tqdm(input_data):
        for paragraph in entry["paragraphs"]:
            if 'arabic' not in paragraph['qas'][0]["id"]:#this will only apply farasa segmentation to Arabic Data
                  continue
            old_context = paragraph["context"]
            paragraph["context"] = clean_preprocess(
                paragraph["context"],
                arabert_prep
            )
            for qas in paragraph["qas"]:
                qas["question"] = clean_preprocess(
                    qas["question"],
                    arabert_prep
                )

                for i in range(len(qas["answers"])):
                  temp_text = clean_preprocess(
                      qas["answers"][i]["text"],
                      arabert_prep
                  )

                  if temp_text == '':
                      temp_text = qas["answers"][i]["text"]

                  answer_location = paragraph["context"].find(
                      temp_text
                  )
                  if answer_location == -1:

                      search_start_pos = get_start_pos(old_context,qas["answers"][i]["answer_start"],arabert_prep)
                      search_end_pos = min(len(paragraph["context"]),search_start_pos+len(temp_text)+20)
                      answer_match =find_near_matches(
                      temp_text,
                      paragraph["context"][search_start_pos:search_end_pos],
                      max_l_dist= min(10,len(temp_text)//2)
                      )
                      if len(answer_match) > 0:
                        tf.logging.warning(
                            "Found new answer for question '%s' :\n '%s' \nvs old.\n '%s'\norig:\n'%s'\ncontext:\n'%s'\n==================",
                            qas["id"],
                            answer_match[i].matched,
                            temp_text,
                            qas["answers"][i]["text"],
                            paragraph["context"]
                        )
                        temp_text = answer_match[i].matched
                        qas["answers"][i]["answer_start"] = answer_match[i].start
                        new_answers_count+=1

                      else:
                        tf.logging.warning(
                            "Could not find answer for question '%s' :\n '%s' \nvs.\n '%s'\norig answer:\n '%s'\n==================",
                            qas["id"],
                            paragraph["context"],
                            temp_text,
                            qas["answers"][i]["text"]
                        )
                        qas["answers"][i]["answer_start"] = -1
                        no_answers_found_count+=1
                  else:
                      qas["answers"][i]["answer_start"] = answer_location

                  if (len(temp_text) + qas["answers"][i]["answer_start"] < (len(paragraph["context"]) + 1)):
                    qas["answers"][i]["text"] = temp_text
                  else:
                    tf.logging.warning(
                            "answer truncated for question '%s' :\n context:\n'%s' \nanswer:\n '%s'\n orig_answer:\n'%s'\nanswer start: %d\nlength of answer: %d\nlength of paragraph: %d\n=================================",
                            qas["id"],
                            paragraph["context"],
                            temp_text,
                            qas["answers"][i]["text"],
                            qas["answers"][0]["answer_start"],
                            len(temp_text),
                            len(paragraph["context"])
                        )
                    qas["answers"][0]["text"] = temp_text[0:len(paragraph["context"])-(len(temp_text) + qas["answers"][0]["answer_start"])]
                    trunc_ans_count+=1

    tf.logging.warning("Found %d new answers: ",new_answers_count)
    tf.logging.warning("Found %d with no answers: ",no_answers_found_count)
    tf.logging.warning("Found %d with trunc answers: ",trunc_ans_count)

    input_data = {
        "data": input_data,
        "version": "1.1",
        "preprocess": "True",
    }
    with tf.gfile.Open(FLAGS.output_file, "w") as writer:
        json.dump(input_data, writer)


if __name__ == "__main__":
    flags.mark_flag_as_required("input_file")
    flags.mark_flag_as_required("output_file")
    flags.mark_flag_as_required("model_name")
    tf.app.run()

Writing prep_arabert_tydiqa.py


In [ ]:
#@title
%%writefile artydiqa.py
"""TODO(tydiqa): Add a description here."""

from __future__ import absolute_import, division, print_function

import json
import textwrap

import datasets


# TODO(tydiqa): BibTeX citation
_CITATION = """\
@article{tydiqa,
title   = {TyDi QA: A Benchmark for Information-Seeking Question Answering in Typologically Diverse Languages},
author  = {Jonathan H. Clark and Eunsol Choi and Michael Collins and Dan Garrette and Tom Kwiatkowski and Vitaly Nikolaev and Jennimaria Palomaki}
year    = {2020},
journal = {Transactions of the Association for Computational Linguistics}
}
"""

# TODO(tydiqa):
_DESCRIPTION = """\
TyDi QA is a question answering dataset covering 11 typologically diverse languages with 204K question-answer pairs.
The languages of TyDi QA are diverse with regard to their typology -- the set of linguistic features that each language
expresses -- such that we expect models performing well on this set to generalize across a large number of the languages
in the world. It contains language phenomena that would not be found in English-only corpora. To provide a realistic
information-seeking task and avoid priming effects, questions are written by people who want to know the answer, but
don’t know the answer yet, (unlike SQuAD and its descendents) and the data is collected directly in each language without
the use of translation (unlike MLQA and XQuAD).
"""

# _URL = "https://storage.googleapis.com/tydiqa/"

# _PRIMARY_URLS = {
#     "train": _URL + "v1.0/tydiqa-v1.0-train.jsonl.gz",
#     "dev": _URL + "v1.0/tydiqa-v1.0-dev.jsonl.gz",
# }
# _SECONDARY_URLS = {
#     "train": _URL + "v1.1/tydiqa-goldp-v1.1-train.json",
#     "dev": _URL + "v1.1/tydiqa-goldp-v1.1-dev.json",
# }

#use this for AraBERTv1 and V2
_URL = "https://storage.googleapis.com/tydiqa/"
_URL2 = "/content/"
_PRIMARY_URLS = {
    "train": _URL + "v1.0/tydiqa-v1.0-train.jsonl.gz",
    "dev": _URL + "v1.0/tydiqa-v1.0-dev.jsonl.gz",
}
_SECONDARY_URLS = {
    "train": _URL2 + "tydiqa-goldp-v1.1-train-pre.json",
    "dev": _URL2 + "tydiqa-goldp-v1.1-dev-pre.json",
}


class TydiqaConfig(datasets.BuilderConfig):

    """ BuilderConfig for Tydiqa"""

    def __init__(self, **kwargs):
        """

        Args:
            **kwargs: keyword arguments forwarded to super.
        """
        super(TydiqaConfig, self).__init__(version=datasets.Version("1.0.0", ""), **kwargs)


class Tydiqa(datasets.GeneratorBasedBuilder):
    """TODO(tydiqa): Short description of my dataset."""

    # TODO(tydiqa): Set up version.
    VERSION = datasets.Version("0.1.0")
    BUILDER_CONFIGS = [
        TydiqaConfig(
            name="primary_task",
            description=textwrap.dedent(
                """\
          Passage selection task (SelectP): Given a list of the passages in the article, return either (a) the index of
          the passage that answers the question or (b) NULL if no such passage exists.
          Minimal answer span task (MinSpan): Given the full text of an article, return one of (a) the start and end
          byte indices of the minimal span that completely answers the question; (b) YES or NO if the question requires
          a yes/no answer and we can draw a conclusion from the passage; (c) NULL if it is not possible to produce a
          minimal answer for this question."""
            ),
        ),
        TydiqaConfig(
            name="secondary_task",
            description=textwrap.dedent(
                """Gold passage task (GoldP): Given a passage that is guaranteed to contain the
          answer, predict the single contiguous span of characters that answers the question. This is more similar to
          existing reading comprehension datasets (as opposed to the information-seeking task outlined above).
          This task is constructed with two goals in mind: (1) more directly comparing with prior work and (2) providing
          a simplified way for researchers to use TyDi QA by providing compatibility with existing code for SQuAD 1.1,
          XQuAD, and MLQA. Toward these goals, the gold passage task differs from the primary task in several ways:
          only the gold answer passage is provided rather than the entire Wikipedia article;
          unanswerable questions have been discarded, similar to MLQA and XQuAD;
          we evaluate with the SQuAD 1.1 metrics like XQuAD; and
         Thai and Japanese are removed since the lack of whitespace breaks some tools.
          """
            ),
        ),
    ]

    def _info(self):
        # TODO(tydiqa): Specifies the datasets.DatasetInfo object
        if self.config.name == "primary_task":
            return datasets.DatasetInfo(
                # This is the description that will appear on the datasets page.
                description=_DESCRIPTION,
                # datasets.features.FeatureConnectors
                features=datasets.Features(
                    {
                        "passage_answer_candidates": datasets.features.Sequence(
                            {
                                "plaintext_start_byte": datasets.Value("int32"),
                                "plaintext_end_byte": datasets.Value("int32"),
                            }
                        ),
                        "question_text": datasets.Value("string"),
                        "document_title": datasets.Value("string"),
                        "language": datasets.Value("string"),
                        "annotations": datasets.features.Sequence(
                            {
                                # 'annotation_id': datasets.Value('variant'),
                                "passage_answer_candidate_index": datasets.Value("int32"),
                                "minimal_answers_start_byte": datasets.Value("int32"),
                                "minimal_answers_end_byte": datasets.Value("int32"),
                                "yes_no_answer": datasets.Value("string"),
                            }
                        ),
                        "document_plaintext": datasets.Value("string"),
                        # 'example_id': datasets.Value('variant'),
                        "document_url": datasets.Value("string")
                        # These are the features of your dataset like images, labels ...
                    }
                ),
                # If there's a common (input, target) tuple from the features,
                # specify them here. They'll be used if as_supervised=True in
                # builder.as_dataset.
                supervised_keys=None,
                # Homepage of the dataset for documentation
                homepage="https://github.com/google-research-datasets/tydiqa",
                citation=_CITATION,
            )
        elif self.config.name == "secondary_task":
            return datasets.DatasetInfo(
                description=_DESCRIPTION,
                features=datasets.Features(
                    {
                        "id": datasets.Value("string"),
                        "title": datasets.Value("string"),
                        "context": datasets.Value("string"),
                        "question": datasets.Value("string"),
                        "answers": datasets.features.Sequence(
                            {
                                "text": datasets.Value("string"),
                                "answer_start": datasets.Value("int32"),
                            }
                        ),
                    }
                ),
                # No default supervised_keys (as we have to pass both question
                # and context as input).
                supervised_keys=None,
                homepage="https://github.com/google-research-datasets/tydiqa",
                citation=_CITATION,
            )

    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        # TODO(tydiqa): Downloads the data and defines the splits
        # dl_manager is a datasets.download.DownloadManager that can be used to
        # download and extract URLs
        primary_downloaded = dl_manager.download_and_extract(_PRIMARY_URLS)
        secondary_downloaded = dl_manager.download_and_extract(_SECONDARY_URLS)
        if self.config.name == "primary_task":
            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["train"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["dev"]},
                ),
            ]
        elif self.config.name == "secondary_task":
            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["train"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["dev"]},
                ),
            ]

    def _generate_examples(self, filepath):
        """Yields examples."""
        # TODO(tydiqa): Yields (key, example) tuples from the dataset
        if self.config.name == "primary_task":
            with open(filepath, encoding="utf-8") as f:
                for id_, row in enumerate(f):
                    data = json.loads(row)
                    passages = data["passage_answer_candidates"]
                    end_byte = [passage["plaintext_end_byte"] for passage in passages]
                    start_byte = [passage["plaintext_start_byte"] for passage in passages]
                    title = data["document_title"]
                    lang = data["language"]
                    question = data["question_text"]
                    annotations = data["annotations"]
                    # annot_ids = [annotation["annotation_id"] for annotation in annotations]
                    yes_no_answers = [annotation["yes_no_answer"] for annotation in annotations]
                    min_answers_end_byte = [
                        annotation["minimal_answer"]["plaintext_end_byte"] for annotation in annotations
                    ]
                    min_answers_start_byte = [
                        annotation["minimal_answer"]["plaintext_start_byte"] for annotation in annotations
                    ]
                    passage_cand_answers = [
                        annotation["passage_answer"]["candidate_index"] for annotation in annotations
                    ]
                    doc = data["document_plaintext"]
                    # example_id = data["example_id"]
                    url = data["document_url"]
                    yield id_, {
                        "passage_answer_candidates": {
                            "plaintext_start_byte": start_byte,
                            "plaintext_end_byte": end_byte,
                        },
                        "question_text": question,
                        "document_title": title,
                        "language": lang,
                        "annotations": {
                            # 'annotation_id': annot_ids,
                            "passage_answer_candidate_index": passage_cand_answers,
                            "minimal_answers_start_byte": min_answers_start_byte,
                            "minimal_answers_end_byte": min_answers_end_byte,
                            "yes_no_answer": yes_no_answers,
                        },
                        "document_plaintext": doc,
                        # 'example_id': example_id,
                        "document_url": url,
                    }
        elif self.config.name == "secondary_task":
            with open(filepath, encoding="utf-8") as f:
                data = json.load(f)
                for article in data["data"]:
                    title = article.get("title", "").strip()
                    for paragraph in article["paragraphs"]:
                        context = paragraph["context"].strip()
                        for qa in paragraph["qas"]:
                            question = qa["question"].strip()
                            id_ = qa["id"]
                            if "arabic" not in id_:
                              continue
                            answer_starts = [answer["answer_start"] for answer in qa["answers"] if answer["answer_start"] != -1]
                            answers = [answer["text"].strip() for answer in qa["answers"] if answer["answer_start"] != -1]
                            if len(answers) == 0 or len(answer_starts)==0:
                              print("question skipped")
                              continue
                            assert len(answer_starts)==len(answers)
                            # Features currently used are "context", "question", and "answers".
                            # Others are extracted here for the ease of future expansions.
                            yield id_, {
                                "title": title,
                                "context": context,
                                "question": question,
                                "id": id_,
                                "answers": {
                                    "answer_start": answer_starts,
                                    "text": answers,
                                },
                            }

Writing artydiqa.py


In [ ]:
#@title
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys
import nltk
import random
nltk.download('punkt')
from random import randint
from pyarabic import araby

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def remove_articles_ar(text):
        text = re.sub('\sال^|ال', ' ', text)
        text = re.sub('^في', ' ', text)
        return text

    def remove_extra_words(text):
        text = text.replace(" ", "")
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation+ "،")
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles_ar(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    scr = (normalize_answer(prediction) == normalize_answer(ground_truth))
    if scr:
      # print("herererere")
      print("Pred:\n",normalize_answer(prediction),"\ntruth:\n", normalize_answer(ground_truth),"\n=======================")
    return scr


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(gold, predictions):
    f1 = exact_match = total = exact_sentence = inclusion = random = 0
    for gold_answer, prediction in zip(gold,predictions):
        # print("Pred:\n",prediction,"\ntruth:\n", gold_answer,"\n=======================")
        total += 1
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, gold_answer)
        f1 += metric_max_over_ground_truths(
            f1_score, prediction, gold_answer)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model_name="aubmindlab/aragpt2-mega"

In [ ]:
!rm -rf *-pre.json
!python prep_arabert_tydiqa.py \
  --input_file "tydiqa-goldp-v1.1-train.json" \
  --output_file "tydiqa-goldp-v1.1-train-pre.json" \
  --model_name=$model_name

2021-02-24 13:53:49.158434: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
aubmindlab/aragpt2-mega
I0224 13:53:50.956272 139944844109696 preprocess.py:125] Selected Model doesn't require pre-segmentation, skipping FarasaSegmenter initialization
 49% 24590/49881 [00:06<00:09, 2687.64it/s]  WARNING:tensorflow:Could not find answer for question 'arabic-1804271180688859213-0' :
 'الإسكندر الثالث المقدوني ، المعروف بأسماء عديدة أخرى أبرزها : الإسكندر الأكبر ، و < b data - parsoid = ' { " dsr " : [1849 , 1870 , 3 , 3] } ' > الإسكندر الكبير ، و < b data - parsoid = ' { " dsr " : [1873 , 1896 , 3 , 3] } ' > الإسكندر المقدوني ، و < b data - parsoid = ' { " dsr " : [1899 , 1924 , 3 , 3] } ' > الإسكندر ذو القرنين ( باليونانية : ؛ نقحرة : ) ، هو أحد ملوك مقدونيا الإغريق ، ومن أشهر القادة العسكريين والفاتحين عبر التاريخ . ولد الإسكندر في مدينة يلا قرابة سنة 356 ق . م ، وتتلمذ على يد الفيلسوف والعالم الشهير أرسطو حتى بلغ ربيعه ال

In [ ]:
!python prep_arabert_tydiqa.py \
  --input_file "tydiqa-goldp-v1.1-dev.json" \
  --output_file "tydiqa-goldp-v1.1-dev-pre.json" \
  --model_name=$model_name

2021-02-24 13:54:14.947973: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
aubmindlab/aragpt2-mega
I0224 13:54:16.689669 139970815948672 preprocess.py:125] Selected Model doesn't require pre-segmentation, skipping FarasaSegmenter initialization
100% 5077/5077 [00:01<00:00, 4522.62it/s]


In [ ]:
from datasets import load_dataset, load_metric
from tqdm.notebook import tqdm
import re

In [ ]:
tydiqa = load_dataset("artydiqa.py",'secondary_task')

question skipped


Dataset tydiqa downloaded and prepared to /root/.cache/huggingface/datasets/tydiqa/secondary_task/1.0.0/3b24f9dbb23b999e16e37526569857be2d6f634d4ef91decdefd4eaf967ebaff. Subsequent calls will reuse this data.


In [ ]:
prompt = """ أجب عن السؤال التالي :"""

In [ ]:
results = []

for example in tqdm(tydiqa['validation']):
  answer = example['answers']['text'][0]
  if example['answers']['answer_start'][0] + len(answer) > 900:
    continue
  question = example['question']
  answer = example['answers']['text'][0]
  context = example['context']

  input_prompt = prompt + " " + question + " الجواب هو" #don't leave a trailing space, because it affects the answer

  answer_pre = re.sub('[^\u0621-\u063A\u0641-\u064A\u0660-\u0669\u0671-\u0673a-zA-Z0-9 ]'," ",answer)
  answer_pre = re.sub('^في عام ', ' ', answer_pre)
  answer_pre = re.sub('^سنة ', ' ', answer_pre)
  answer_pre = re.sub('^في ', ' ', answer_pre)
  answer_pre = re.sub(' م$', ' ', answer_pre)
  answer_pre = re.sub(' ه$', ' ', answer_pre)
  answer_pre = ' '.join(answer_pre.split()).strip()

  if answer_pre != '':
    answer= answer_pre
  for word in answer.split():
    if word.isnumeric():
      if 'متى' in question or 'عام' in question or 'سنة' in question:
        input_prompt+= " في عام"
        break

  input_prompt = ' '.join(input_prompt.split()).strip()

  input_tok = tokenizer.tokenize(input_prompt)
  answer_tok = tokenizer.tokenize(answer)

  gen_text = aragpt2_pipeline(input_prompt,
            pad_token_id=0, # 0 for AraGPT2
            do_sample = False,
            num_beams=5,
            max_length=int(len(input_tok) + len(answer_tok)),
            top_k=10,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3,
            num_return_sequences = 5
            )[0]['generated_text']

  pred = gen_text[len(input_prompt):]

  results.append(
      {
        'context': context,
        'question': question,
        'prompt': input_prompt,
        'pred_answer': pred,
        'answer' : answer,
      }
  )

In [ ]:
results[10]

{'answer': 'أدولف هتلر',
 'context': 'Coordinates : ألمانيا النازية هو اسم شائع غير رسمي يطلق على ألمانيا في الفترة الممتدة من عام 1933 حتى عام 1945 ، نسبة الى اسم الحزب النازي عندما كان أدولف هتلر وحزبه يحكمان ألمانيا دكتاتوريا . تحولت ألمانيا - تحت حكم نظام هتلر - إلى دولة فاشية استولى فيها الحزب النازي على البلاد وسيطر بشمولية على جميع جوانب الحياة تقريبا . بينما كان الاسم الرسمي للدولة الرايخ الألماني " ديوتشس رايش " من سنة 1933 حتى 1943 و " غروديوتشس رايش " الرايخ الألماني الأكبر من 1943 إلى 1945 . هذه الفترة معروفة أيضا باسم أسماء الرايخ الثالث ( الألمانية : دريتس رايش ) أو الفترة الاشتراكية الوطنية ( الألمانية : زيت ديس ناتيونسوزياليسموس ، مختصر باسم نس - زيت ) . انتهى النظام النازي بعد أن هزمت ألمانيا في مايو 1945 ، مما أنهى الحرب العالمية الثانية في أوروبا .',
 'pred_answer': ' أدولف هتلر .',
 'prompt': 'أجب عن السؤال التالي : من رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ الجواب هو',
 'question': 'من رئيس ألمانيا النازية في الحرب العالمية الثانية ؟'}

In [ ]:
gold = [[x['answer']] for x in results]
preds = [ x['pred_answer'] for x in results]

In [ ]:
with open("results.txt",'w',encoding='utf-8') as f:
  for r in results:
    print("context : ",r['context'])
    print("question : ",r['question'])
    print("prompt : ",r['prompt'])
    print("pred_answer : ",r['pred_answer'])
    print("answer : ",r['answer'])
    print("\n")

    f.write("context : ")
    f.write(r['context']+'\n')
    f.write("question : ")
    f.write(r['question']+'\n')
    f.write("prompt : ")
    f.write(r['prompt']+'\n')
    f.write("pred_answer : ")
    f.write(r['pred_answer']+'\n')
    f.write("answer : ")
    f.write(r['answer']+'\n')
    f.write("\n")

In [ ]:
scores = evaluate(gold, preds)
print(scores)

Pred:
 تبليسي 
truth:
 تبليسي 
Pred:
 أدولف هتلر 
truth:
 أدولف هتلر 
Pred:
 1962 
truth:
 1962 
Pred:
 مابوتو 
truth:
 مابوتو 
Pred:
 تبليسي 
truth:
 تبليسي 
Pred:
 2014 
truth:
 2014 
Pred:
 فرنسا 
truth:
 فرنسا 
Pred:
 1890 
truth:
 1890 
Pred:
 يابان 
truth:
 يابان 
Pred:
 1946 
truth:
 1946 
Pred:
 ابن خلدون 
truth:
 ابن خلدون 
Pred:
 2002 
truth:
 2002 
Pred:
 1911 
truth:
 1911 
Pred:
 1962 
truth:
 1962 
Pred:
 مضيق دردنيل 
truth:
 مضيق دردنيل 
Pred:
 أبو محمد جولاني 
truth:
 أبو محمد جولاني 
Pred:
 1867 
truth:
 1867 
Pred:
 قابيل وهابيل 
truth:
 قابيل وهابيل 
Pred:
 1979 
truth:
 1979 
Pred:
 ك يفورنيا 
truth:
 ك يفورنيا 
Pred:
 هراري 
truth:
 هراري 
Pred:
 1955 
truth:
 1955 
Pred:
 لغة ألمانية 
truth:
 لغة ألمانية 
Pred:
 1981 
truth:
 1981 
Pred:
 2014 
truth:
 2014 
Pred:
 دينار عراقي 
truth:
 دينار عراقي 
Pred:
 مكة مكرمة 
truth:
 مكة مكرمة 
Pred:
 يثرب 
truth:
 يثرب 
Pred:
 محمد نجيب 
truth:
 محمد نجيب 
Pred:
 1997 
truth:
 1997 
Pred:
 جامعة سنغافورة وطنية 
truth:
 جام

Manual Inspection of the first 500 answers, shows that AraGPT2 actually answered **24.55%** of the questions correctly. Since some of the prediction didn't get an exact match score.

In [ ]:
for gold_answer, prediction in zip(gold,preds):
    print("Pred:\n",prediction,"\ntruth:\n", gold_answer[0],"\n=======================")

# Zero-Shot question answering - ARCD

Start testing some prompts, to see what works

In [ ]:
prompt = """
أجب عن السؤال التالي:
"""

In [ ]:
text = """
\"من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟\"
"""
text_tok = tokenizer.tokenize(arabert_processor.preprocess(text))
text_len = len(text_tok)
print(text_tok)
print(text_len)

['"', 'ĠÙħÙĨ', 'ĠÙĥØ§ÙĨ', 'ĠØ±Ø¦ÙĬØ³', 'ĠØ£ÙĦÙħØ§ÙĨÙĬØ§', 'ĠØ§ÙĦÙĨØ§Ø²ÙĬØ©', 'ĠÙģÙĬ', 'ĠØ§ÙĦØŃØ±Ø¨', 'ĠØ§ÙĦØ¹Ø§ÙĦÙħÙĬØ©', 'ĠØ§ÙĦØ«Ø§ÙĨÙĬØ©', 'ĠØŁ', 'Ġ"']
12


In [ ]:
text_prep = arabert_processor.preprocess(prompt + " " + text)
input_len = len(tokenizer.tokenize(text_prep))
print(text_prep)
print(input_len)

أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ "
18


In [ ]:
print("Input Length: ", input_len + text_len //2)
gen_text = aragpt2_pipeline(text_prep,
            pad_token_id=0, # 0 for AraGPT2
            do_sample = False,
            num_beams=5,
            max_length=input_len + 10,
            top_k=10,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3,
            num_return_sequences = 5)
gen_text

Input Length:  24


[{'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " الجواب هو أدولف هتلر ، الذي حكم ألمانيا بين'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " الجواب هو أدولف هتلر ، الذي حكم ألمانيا لمدة'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " الجواب هو أدولف هتلر ، الذي حكم ألمانيا منذ'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " إن الإجابة على هذا السؤال صعبة جدا ، لأن هتلر'},
 {'generated_text': 'أجب عن السؤال التالي : " من كان رئيس ألمانيا النازية في الحرب العالمية الثانية ؟ " إن الإجابة على هذا السؤال صعبة للغاية ، لأن هتلر'}]

Once ready, download the ARCD test data to evaluate the prompt

In [ ]:
!wget https://raw.githubusercontent.com/husseinmozannar/SOQAL/master/data/arcd-test.json

--2021-02-24 15:20:09--  https://raw.githubusercontent.com/husseinmozannar/SOQAL/master/data/arcd-test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002559 (979K) [text/plain]
Saving to: ‘arcd-test.json’

arcd-test.json      100%[===================>] 979.06K  --.-KB/s    in 0.03s   

2021-02-24 15:20:09 (30.7 MB/s) - ‘arcd-test.json’ saved [1002559/1002559]



In [ ]:
#@title
%%writefile arcd_preprocessing.py
# coding=utf-8
#
# This script applies AraBERT's cleaning process and segmentation to ARCD or
# any SQUAD-like structured files and "naively" re-alligns the answers start positions

import tensorflow.compat.v1 as tf
from arabert.preprocess import ArabertPreprocessor
from arabert.arabert.tokenization import BasicTokenizer
from tqdm import tqdm
import json

from fuzzysearch import find_near_matches
from pyarabic import araby

flags = tf.flags
FLAGS = flags.FLAGS


## Required parameters
flags.DEFINE_string(
    "input_file", None, "The input json file with a SQUAD like structure."
)

flags.DEFINE_string(
    "output_file", None, "The ouput json file with AraBERT preprocessing applied."
)

flags.DEFINE_string(
    "model_name", None, "Model name same as HuggingFace library"
)


bt = BasicTokenizer(do_lower_case=False)

def clean_preprocess(text, processor):
    text = " ".join(
        bt._run_split_on_punc(text)
    )
    text= processor.preprocess(text)
    text = " ".join(text.split())  # removes extra whitespaces
    return text

def get_start_pos(old_context, old_answer_start,processor):
  new_context = clean_preprocess(old_context[:old_answer_start],processor)
  num_of_pluses = new_context.count("+")
  return old_answer_start + num_of_pluses*2 - 20

def normalize_alef(s):
    s = s.replace(araby.ALEF_HAMZA_ABOVE,araby.ALEF)
    s = s.replace(araby.ALEF_HAMZA_BELOW,araby.ALEF)
    s = s.replace(araby.ALEF_MADDA,araby.ALEF)
    return s

def main(_):
    tf.logging.set_verbosity(tf.logging.INFO)
    logger = tf.get_logger()
    logger.propagate = False

    print(FLAGS.model_name)
    arabert_prep = ArabertPreprocessor(model_name=FLAGS.model_name, remove_html_markup=False)


    with tf.gfile.Open(FLAGS.input_file, "r") as reader:
        input_data = json.load(reader)["data"]

    new_answers_count = 0
    no_answers_found_count = 0
    trunc_ans_count = 0
    for entry in tqdm(input_data):
        for paragraph in entry["paragraphs"]:
            old_context = paragraph["context"]
            paragraph["context"] = clean_preprocess(
                paragraph["context"],
                arabert_prep
            )
            for qas in paragraph["qas"]:
                qas["question"] = clean_preprocess(
                    qas["question"],
                    arabert_prep
                )

                for i in range(len(qas["answers"])):
                  temp_text = clean_preprocess(
                      qas["answers"][i]["text"],
                      arabert_prep
                  )

                  if temp_text == '':
                      temp_text = qas["answers"][i]["text"]

                  answer_location = paragraph["context"].find(
                      temp_text
                  )
                  if answer_location == -1:

                      search_start_pos = get_start_pos(old_context,qas["answers"][i]["answer_start"],arabert_prep)
                      search_end_pos = min(len(paragraph["context"]),search_start_pos+len(temp_text)+20)
                      answer_match =find_near_matches(
                      temp_text,
                      paragraph["context"][search_start_pos:search_end_pos],
                      max_l_dist= min(10,len(temp_text)//2)
                      )
                      if len(answer_match) > 0:
                        tf.logging.warning(
                            "Found new answer for question '%s' :\n '%s' \nvs old.\n '%s'\norig:\n'%s'\ncontext:\n'%s'\n==================",
                            qas["id"],
                            answer_match[i].matched,
                            temp_text,
                            qas["answers"][i]["text"],
                            paragraph["context"]
                        )
                        temp_text = answer_match[i].matched
                        qas["answers"][i]["answer_start"] = answer_match[i].start
                        new_answers_count+=1

                      else:
                        tf.logging.warning(
                            "Could not find answer for question '%s' :\n '%s' \nvs.\n '%s'\norig answer:\n '%s'\n==================",
                            qas["id"],
                            paragraph["context"],
                            temp_text,
                            qas["answers"][i]["text"]
                        )
                        qas["answers"][i]["answer_start"] = -1
                        no_answers_found_count+=1
                  else:
                      qas["answers"][i]["answer_start"] = answer_location

                  if (len(temp_text) + qas["answers"][i]["answer_start"] < (len(paragraph["context"]) + 1)):
                    qas["answers"][i]["text"] = temp_text
                  else:
                    tf.logging.warning(
                            "answer truncated for question '%s' :\n context:\n'%s' \nanswer:\n '%s'\n orig_answer:\n'%s'\nanswer start: %d\nlength of answer: %d\nlength of paragraph: %d\n=================================",
                            qas["id"],
                            paragraph["context"],
                            temp_text,
                            qas["answers"][i]["text"],
                            qas["answers"][0]["answer_start"],
                            len(temp_text),
                            len(paragraph["context"])
                        )
                    qas["answers"][0]["text"] = temp_text[0:len(paragraph["context"])-(len(temp_text) + qas["answers"][0]["answer_start"])]
                    trunc_ans_count+=1

    tf.logging.warning("Found %d new answers: ",new_answers_count)
    tf.logging.warning("Found %d with no answers: ",no_answers_found_count)
    tf.logging.warning("Found %d with trunc answers: ",trunc_ans_count)

    input_data = {
        "data": input_data,
        "version": "1.1",
        "preprocess": "True",
    }
    with tf.gfile.Open(FLAGS.output_file, "w") as writer:
        json.dump(input_data, writer)


if __name__ == "__main__":
    flags.mark_flag_as_required("input_file")
    flags.mark_flag_as_required("output_file")
    flags.mark_flag_as_required("model_name")
    tf.app.run()

Writing arcd_preprocessing.py


In [ ]:
#@title
%%writefile arcd_arabert.py
"""ARCD: Arabic Reading Comprehension Dataset."""

from __future__ import absolute_import, division, print_function

import json
import logging

import datasets


_CITATION = """\
@inproceedings{mozannar-etal-2019-neural,
    title = {Neural {A}rabic Question Answering},
    author = {Mozannar, Hussein  and Maamary, Elie  and El Hajal, Karl  and Hajj, Hazem},
    booktitle = {Proceedings of the Fourth Arabic Natural Language Processing Workshop},
    month = {aug},
    year = {2019},
    address = {Florence, Italy},
    publisher = {Association for Computational Linguistics},
    url = {https://www.aclweb.org/anthology/W19-4612},
    doi = {10.18653/v1/W19-4612},
    pages = {108--118},
    abstract = {This paper tackles the problem of open domain factual Arabic question answering (QA) using Wikipedia as our knowledge source. This constrains the answer of any question to be a span of text in Wikipedia. Open domain QA for Arabic entails three challenges: annotated QA datasets in Arabic, large scale efficient information retrieval and machine reading comprehension. To deal with the lack of Arabic QA datasets we present the Arabic Reading Comprehension Dataset (ARCD) composed of 1,395 questions posed by crowdworkers on Wikipedia articles, and a machine translation of the Stanford Question Answering Dataset (Arabic-SQuAD). Our system for open domain question answering in Arabic (SOQAL) is based on two components: (1) a document retriever using a hierarchical TF-IDF approach and (2) a neural reading comprehension model using the pre-trained bi-directional transformer BERT. Our experiments on ARCD indicate the effectiveness of our approach with our BERT-based reader achieving a 61.3 F1 score, and our open domain system SOQAL achieving a 27.6 F1 score.}
}
"""

_DESCRIPTION = """\
 Arabic Reading Comprehension Dataset (ARCD) composed of 1,395 questions\
      posed by crowdworkers on Wikipedia articles.
"""

_URL = "./"
_URLs = {
    "train": _URL + "arcd-test-pre.json",
    "dev": _URL + "arcd-test-pre.json",
}



class ArcdAraBERTConfig(datasets.BuilderConfig):
    """BuilderConfig for ARCD."""

    def __init__(self, **kwargs):
        """BuilderConfig for ARCD.

        Args:
          **kwargs: keyword arguments forwarded to super.
        """
        super(ArcdAraBERTConfig, self).__init__(**kwargs)


class ArcdAraBERT(datasets.GeneratorBasedBuilder):
    """ARCD: Arabic Reading Comprehension Dataset."""

    BUILDER_CONFIGS = [
        ArcdAraBERTConfig(
            name="plain_text",
            version=datasets.Version("1.0.0", ""),
            description="Plain text",
        )
    ]

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "title": datasets.Value("string"),
                    "context": datasets.Value("string"),
                    "question": datasets.Value("string"),
                    "answers": datasets.features.Sequence(
                        {"text": datasets.Value("string"), "answer_start": datasets.Value("int32")}
                    ),
                }
            ),
            # No default supervised_keys (as we have to pass both question
            # and context as input).
            supervised_keys=None,
            homepage="https://github.com/husseinmozannar/SOQAL/tree/master/data",
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        urls_to_download = _URLs
        downloaded_files = dl_manager.download_and_extract(urls_to_download)

        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"filepath": downloaded_files["train"]}),
            datasets.SplitGenerator(name=datasets.Split.VALIDATION, gen_kwargs={"filepath": downloaded_files["dev"]}),
        ]

    def _generate_examples(self, filepath):
        """This function returns the examples in the raw (text) form."""
        logging.info("generating examples from = %s", filepath)
        with open(filepath, encoding="utf-8") as f:
            arcd = json.load(f)
            for article in arcd["data"]:
                title = article.get("title", "").strip()
                for paragraph in article["paragraphs"]:
                    context = paragraph["context"].strip()
                    for qa in paragraph["qas"]:
                        question = qa["question"].strip()

                        id_ = qa["id"]

                        answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                        answers = [answer["text"].strip() for answer in qa["answers"]]

                        # Features currently used are "context", "question", and "answers".
                        # Others are extracted here for the ease of future expansions.
                        yield id_, {
                            "title": title,
                            "context": context,
                            "question": question,
                            "id": id_,
                            "answers": {"answer_start": answer_starts, "text": answers},
                        }

Writing arcd_arabert.py


In [ ]:
#@title
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys
import nltk
import random
nltk.download('punkt')
from random import randint
from pyarabic import araby

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def remove_articles_ar(text):
        text = re.sub('\sال^|ال', ' ', text)
        text = re.sub('^في', ' ', text)
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation+ "،")
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles_ar(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    scr = (normalize_answer(prediction) == normalize_answer(ground_truth))
    if scr:
      # print("herererere")
      print("Pred:\n",normalize_answer(prediction),"\ntruth:\n", normalize_answer(ground_truth),"\n=======================")
    return scr


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(gold, predictions):
    f1 = exact_match = total = exact_sentence = inclusion = random = 0
    for gold_answer, prediction in zip(gold,predictions):
        # print("Pred:\n",prediction,"\ntruth:\n", gold_answer,"\n=======================")
        total += 1
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, gold_answer)
        f1 += metric_max_over_ground_truths(
            f1_score, prediction, gold_answer)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model_name="aubmindlab/aragpt2-mega"

In [ ]:
!rm -rf *-pre.json
!python arcd_preprocessing.py \
  --input_file="/content/arcd-test.json" \
  --output_file="/content/arcd-test-pre.json" \
  --model_name=$model_name

2021-02-24 15:20:13.241405: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
aubmindlab/aragpt2-mega
I0224 15:20:15.302043 140188850329472 preprocess.py:125] Selected Model doesn't require pre-segmentation, skipping FarasaSegmenter initialization
100% 78/78 [00:00<00:00, 188.99it/s]


In [ ]:
from datasets import load_dataset, load_metric
from tqdm.notebook import tqdm
import re

In [ ]:
arcd = load_dataset("arcd_arabert.py")

Dataset arcd_ara_bert downloaded and prepared to /root/.cache/huggingface/datasets/arcd_ara_bert/plain_text/1.0.0/3a414eb9e17716bbfc3a2ff0c6494ee61437f371125e8f9b176b704e05f872d8. Subsequent calls will reuse this data.


In [ ]:
prompt = """ أجب عن السؤال التالي :"""

In [ ]:
results = []

for example in tqdm(arcd['validation']):
  answer = example['answers']['text'][0]
  if example['answers']['answer_start'][0] + len(answer) > 900:
    continue
  question = example['question']
  answer = example['answers']['text'][0]
  context = example['context']

  input_prompt = prompt + " " + question + " الجواب هو" #don't leave a trailing space, because it affects the answer

  answer_pre = re.sub('[^\u0621-\u063A\u0641-\u064A\u0660-\u0669\u0671-\u0673a-zA-Z0-9 ]'," ",answer)
  answer_pre = re.sub('^في عام ', ' ', answer_pre)
  answer_pre = re.sub('^سنة ', ' ', answer_pre)
  answer_pre = re.sub('^في ', ' ', answer_pre)
  answer_pre = re.sub(' م$', ' ', answer_pre)
  answer_pre = ' '.join(answer_pre.split()).strip()

  if answer_pre != '':
    answer= answer_pre
  if answer.isnumeric():
    input_prompt+= " في عام"

  input_prompt = ' '.join(input_prompt.split()).strip()

  input_tok = tokenizer.tokenize(input_prompt)
  answer_tok = tokenizer.tokenize(answer)

  gen_text = aragpt2_pipeline(input_prompt,
            pad_token_id=0, # 0 for AraGPT2
            do_sample = False,
            num_beams=5,
            max_length=int(len(input_tok) + len(answer_tok)),
            top_k=10,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3,
            num_return_sequences = 5
            )[0]['generated_text']

  pred = gen_text[len(input_prompt):]

  results.append(
      {
        'answer' : answer,
        'pred_answer': pred,
        'question': question,
        'context': context,
        'prompt': input_prompt
      }
  )

In [ ]:
gold = [[x['answer']] for x in results]
preds = [ x['pred_answer'] for x in results]

In [ ]:
with open("results.txt",'w',encoding='utf-8') as f:
  for r in results:
    print("context : ",r['context'])
    print("question : ",r['question'])
    print("prompt : ",r['prompt'])
    print("pred_answer : ",r['pred_answer'])
    print("answer : ",r['answer'])
    print("\n")

    f.write("context : ")
    f.write(r['context']+'\n')
    f.write("question : ")
    f.write(r['question']+'\n')
    f.write("prompt : ")
    f.write(r['prompt']+'\n')
    f.write("pred_answer : ")
    f.write(r['pred_answer']+'\n')
    f.write("answer : ")
    f.write(r['answer']+'\n')
    f.write("\n")

context :  حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحابة رسول الإسلام محمد ، وعمه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر ، وهو خير أعمامه لقوله : « خير إخوتي علي ، وخير أعمامي حمزة رضي الله عنهما » .
question :  من هو حمزة بن عبد المطلب ؟
prompt :  أجب عن السؤال التالي : من هو حمزة بن عبد المطلب ؟ الجواب هو
pred_answer :   : أبو طالب عم النبي صلى الله عليه وسلم ، وهو أول من آمن به وصدقه قبل
answer :  صحابي من صحابة رسول الإسلام محمد وعمه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر


context :  حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحابة رسول الإسلام محمد ، وعمه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر ، وهو خير أعمامه لقوله : « خير إخوتي علي ، وخير أعمامي حمزة رضي الله عنهما » .
question :  بما وصفه رسول الله ؟
prompt :  أجب عن السؤال التالي : بما وصفه رسول الله ؟ الجواب هو
pred_answer :   : نعم ، لقد
answer :  وخير أعمامي


context :  حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحابة رسول الإسلام محمد ، وعمه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر ، وهو خير أعمامه لقوله : « خير

In [ ]:
scores = evaluate(gold, preds)
print(scores)

Pred:
  
truth:
  
Pred:
 جنكيز خان 
truth:
 جنكيز خان 
Pred:
 خ د بن وليد 
truth:
 خ د بن وليد 
Pred:
 1956 
truth:
 1956 
Pred:
 1970 
truth:
 1970 
Pred:
 مدينة كويت 
truth:
 مدينة كويت 
Pred:
 خمس مرات 
truth:
 خمس مرات 
Pred:
 اتحاد دولي لكرة قدم فيفا 
truth:
 اتحاد دولي لكرة قدم فيفا 
Pred:
 1921 
truth:
 1921 
Pred:
 1946 
truth:
 1946 
Pred:
 نظام ملكي دستوري 
truth:
 نظام ملكي دستوري 
Pred:
 هندوسية و بوذية و جاينية و سيخية 
truth:
 هندوسية و بوذية و جاينية و سيخية 
Pred:
 1947 
truth:
 1947 
Pred:
 هيرودوت 
truth:
 هيرودوت 
Pred:
 قرآن كريم 
truth:
 قرآن كريم 
Pred:
 له 
truth:
 له 
Pred:
 1991 
truth:
 1991 
Pred:
 1956 
truth:
 1956 
Pred:
 2011 
truth:
 2011 
Pred:
 1892 
truth:
 1892 
Pred:
 ملعب أنفيلد 
truth:
 ملعب أنفيلد 
Pred:
 سل ة تشينغ 
truth:
 سل ة تشينغ 
Pred:
 1949 
truth:
 1949 
Pred:
 حارس مرمى 
truth:
 حارس مرمى 
Pred:
 1863 
truth:
 1863 
Pred:
 عام 1991 
truth:
 عام 1991 
Pred:
 فرنسا 
truth:
 فرنسا 
Pred:
 2005 
truth:
 2005 
{'exact_match': 4.075691411935

In [ ]:
for gold_answer, prediction in zip(gold,preds):
    print("Pred:\n",prediction,"\ntruth:\n", gold_answer[0],"\n=======================")

Pred:
  : أبو طالب عم النبي صلى الله عليه وسلم ، وهو أول من آمن به وصدقه قبل 
truth:
 صحابي من صحابة رسول الإسلام محمد وعمه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر 
Pred:
  : نعم ، لقد 
truth:
 وخير أعمامي 
Pred:
  ما جاء في الحديث 
truth:
 خير إخوتي علي 
Pred:
  ان حمزة بن عبد المطلب أسلم عام 
truth:
 السنة الثانية من بعثة النبي محمد 
Pred:
  أن رسول الله صلى الله عليه و سلم لم يقاتل أحدا إلا بعد 
truth:
 وقتل فيها شيبة بن ربيعة مبارزة وقتل غيره كثيرا من المشركين 
Pred:
  : معاوية بن أبي سفيان . 
truth:
 وحشي بن حرب الحبشي 
Pred:
  أن عبد المطلب كان 
truth:
 سيد قريش وبني هاشم 
Pred:
  محمد بن عبد الله صلى الله عليه 
truth:
 محمد بن عبد الله بن عبد المطلب 
Pred:
  ما يلي : ( 1 ) 
truth:
 صداقة متينة ووثيقة العرا 
Pred:
  أن هناك ثلاثة أنواع من 
truth:
 القمر الطبيعي الوحيد للأرض 
Pred:
  1 . 5 
truth:
 ربع قطر الأرض 
Pred:
  واحد على عشرة آلاف من كتلة 
truth:
 1 على 81 من كتلة الأرض 
Pred:
  لوكالة الفضاء الامريكية ناسا 
truth:
 للإتحاد السوفيتي 
Pred:
  1969  
truth:
 1968 
Pred:
  